In [1]:
import deepchem as dc
import conda_installer
import matplotlib.pyplot as plt
import numpy as np
# conda_installer.install()
conda_installer.install()
# !/root/miniconda/bin/conda info -e
import rdkit

2023-01-18 04:43:50.057661: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-18 04:43:50.057700: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
add /home/ali/miniconda/lib/python3.8/site-packages to PYTHONPATH
all packages are already installed


In [26]:
smiles = ["CCC", "N"]
import importlib
importlib.reload(dc)
featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)
f = featurizer.featurize(smiles)
mols = f
max_deg = 10
min_deg = 0
num_mols = 2

In [27]:
atoms_by_deg = np.concatenate([x.atom_features for x in f])
degree_vector = np.concatenate([x.degree_list for x in f], axis=0)

In [33]:
degree_vector[order]

array([0, 1, 1, 2])

In [6]:
order = degree_vector.argsort(kind='mergesort')
ordered = np.empty(order.shape, np.int32)

In [16]:
order

array([3, 0, 1, 2])

In [18]:
ordered

array([1, 2, 3, 0], dtype=int32)

In [8]:
ordered[order] = np.arange(order.shape[0], dtype=np.int32)

In [19]:
mol_atom_map = []
index_start = 0
for mol in mols:
    mol_atom_map.append(
      ordered[index_start:index_start + mol.get_num_atoms()])
    index_start += mol.get_num_atoms()

In [34]:
mol_atom_map

[array([1, 2, 3], dtype=int32), array([0], dtype=int32)]

In [28]:
mol_deg_sz = np.zeros([max_deg - min_deg + 1, num_mols], dtype=np.int32)
for i, mol in enumerate(mols):
    mol_deg_sz[:, i] += mol.deg_slice[:, 1]

In [29]:
mol_deg_sz

array([[0, 1],
       [2, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int32)

In [35]:
from deepchem.feat.mol_graphs import ConvMol

In [49]:
multiConvMol = ConvMol.agglomerate_mols(f)

In [52]:
inputs = [
            multiConvMol.get_atom_features(), multiConvMol.deg_slice,
            np.array(multiConvMol.membership), 2
        ]
for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
    inputs.append(multiConvMol.get_deg_adjacency_lists()[i])

In [60]:
import tensorflow as tf
from deepchem.models.keras_model import KerasModel
model = KerasModel(tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
]), loss=dc.models.losses.L2Loss())
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# model.compile(optimizer='adam',
#               loss=loss_fn,
#               metrics=['accuracy'])

In [62]:
model.fit(inputs, np.array([1, 1]))

TypeError: only integer scalar arrays can be converted to a scalar index